In [98]:
from fastquant import get_stock_data
from gaussian_hmm import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import time
import sys
import os

In [99]:
with open('pass.txt') as f:
    password = f.read()
    password = password.strip('\n')

In [100]:
f = open("tickers.txt", "r")
tickers = [s.strip('\n') for s in f.readlines()]

In [101]:
tickers

['AAPL', 'TSLA', 'TSM', 'HD', 'PLUG', 'DKNG', 'CMCSA', 'PFE']

In [30]:
for ticker in tickers:
    print(get_stock_data(ticker, '2017-01-01', '2021-05-12'))

[*********************100%***********************]  1 of 1 completed
                  open        high         low       close     volume
dt                                                                   
2017-01-03   28.950001   29.082500   28.690001   29.037500  115127600
2017-01-04   28.962500   29.127501   28.937500   29.004999   84472400
2017-01-05   28.980000   29.215000   28.952499   29.152500   88774400
2017-01-06   29.195000   29.540001   29.117500   29.477501  127007600
2017-01-09   29.487499   29.857500   29.485001   29.747499  134247600
...                ...         ...         ...         ...        ...
2021-05-05  129.199997  130.449997  127.970001  128.100006   84000900
2021-05-06  127.889999  129.750000  127.129997  129.740005   78128300
2021-05-07  130.850006  131.259995  129.479996  130.210007   78892700
2021-05-10  129.410004  129.539993  126.809998  126.849998   87808400
2021-05-11  123.500000  125.724998  122.779999  125.305496   80260673

[1096 rows x 5 colum

In [31]:
import smtplib

In [88]:
def send_email(sent_to, message):
    sender = 'ghmm.stock.bot@gmail.com'

    gmail_user = 'ghmm.stock.bot@gmail.com'
    sent_from = 'ghmm.stock.bot@gmail.com'
    email_text = message

    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login(gmail_user, password)
    server.sendmail(sent_from, sent_to, email_text)
    server.close()

In [83]:
recievers = [('rowan','ghmm.stock.bot@gmail.com')]

In [33]:
from datetime import datetime, timedelta

In [34]:
quant_format = '%Y-%m-%d'

In [36]:
today = datetime.now()

In [37]:
training_start = today - timedelta(days=1825)

In [38]:
training_start

datetime.datetime(2016, 5, 12, 12, 38, 25, 646614)

In [39]:
today

datetime.datetime(2021, 5, 11, 12, 38, 25, 646614)

In [40]:
training_end = today - timedelta(days=1)

In [41]:
training_start = training_start.strftime(quant_format)

In [42]:
training_start

'2016-05-12'

In [43]:
training_end = training_end.strftime(quant_format)

In [44]:
training_end

'2021-05-10'

In [45]:
tomorrow = today + timedelta(days=1)

In [46]:
today = today.strftime(quant_format)

In [47]:
tomorrow = tomorrow.strftime(quant_format)

In [48]:
training_start

'2016-05-12'

In [50]:
today

'2021-05-11'

In [51]:
tomorrow

'2021-05-12'

In [52]:
training_data = get_stock_data('AAPL', training_start, today)

[*********************100%***********************]  1 of 1 completed


In [53]:
test_data = get_stock_data('AAPL', today, tomorrow)

[*********************100%***********************]  1 of 1 completed


In [54]:
training_data

,open,high,low,close,volume
dt,,,,,
2016-05-12,23.180000,23.195000,22.367500,22.584999,305258800
2016-05-13,22.500000,22.917500,22.500000,22.629999,177571200
2016-05-16,23.097500,23.597500,22.912500,23.469999,245039200
2016-05-17,23.637501,23.674999,23.252501,23.372499,187667600
2016-05-18,23.540001,23.802500,23.472500,23.639999,168249600
...,...,...,...,...,...
2021-05-04,131.190002,131.490005,126.699997,127.849998,137564700
2021-05-05,129.199997,130.449997,127.970001,128.100006,84000900
2021-05-06,127.889999,129.750000,127.129997,129.740005,78128300


In [55]:
test_data

,open,high,low,close,volume
dt,,,,,
2021-05-11,123.5,125.724998,122.779999,125.129997,81844688


In [56]:
params = {'n_components': 2, 
          'algorithm': 'map', 
          'n_iter': 100, 
          'd': 5,
          'name':'GHMM'}
ghmm = GHMM(params=params)

In [57]:
ghmm.train(training_data)

In [58]:
preds,_ = ghmm.predict(test_data)

DONE


In [59]:
preds

[123.59577042192016]

In [60]:
prev_close = training_data['close'].values[-1]

In [61]:
pred_close = preds[0]

In [62]:
change = (pred_close-prev_close)/prev_close

In [63]:
change

-0.025654143408325152

In [64]:
open_price = test_data['open'].values[0]

In [65]:
frac_change = (pred_close-open_price)/open_price

In [66]:
frac_change

0.0007754690034021416

In [129]:
def get_data(ticker):
    today = datetime.now()
    training_start = today - timedelta(days=1825)
    training_start = training_start.strftime(quant_format)
    tomorrow = today + timedelta(days=1)
    today = today.strftime(quant_format)
    tomorrow = tomorrow.strftime(quant_format)
    training_data = get_stock_data(ticker, training_start, today)
    test_data = get_stock_data(ticker, today, tomorrow)
    return training_data, test_data

In [130]:
def get_prediction(ticker):
    params = {'n_components': 2, 
          'algorithm': 'map', 
          'n_iter': 100, 
          'd': 5,
          'name':'GHMM'}
    ghmm = GHMM(params=params)
    train,test = get_data(ticker)
    ghmm.train(train)
    pred,_ = ghmm.predict(test)
    open_price = test['open'].values[0]
    pred_close = pred[0]
    frac_change = (pred_close-open_price)/open_price
    return frac_change, open_price, pred_close

In [131]:
sender = 'ghmm.stock.bot@gmail.com'
start = time.time()
picks = ""
for ticker in tickers:
    c,o,p = get_prediction(ticker)
    picks += f'Ticker: {ticker}\nOpen: {round(o,3)}\nPred Close: {round(p,3)}\nPred Percent Change: {round(c*100,3)}\n\n'
end = time.time()
print(round(end-start,4))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
DONE
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
DONE
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
DONE
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
DONE
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
DONE
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
DONE
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
DONE

In [132]:
for name,email in recievers:
        message = f'From: From Rowan <{sender}>\nTo: To {name} <{email}>\nSubject: Daily Picks\n\n{picks}'
        
        print(message)
        send_email(email,message)

From: From Rowan <ghmm.stock.bot@gmail.com>
To: To rowan <ghmm.stock.bot@gmail.com>
Subject: Daily Picks

Ticker: AAPL
Open: 123.5
Pred Close: 123.596
Pred Percent Change: 0.078

Ticker: TSLA
Open: 599.24
Pred Close: 599.824
Pred Percent Change: 0.097

Ticker: TSM
Open: 109.73
Pred Close: 109.758
Pred Percent Change: 0.025

Ticker: HD
Open: 337.55
Pred Close: 337.738
Pred Percent Change: 0.056

Ticker: PLUG
Open: 18.6
Pred Close: 18.595
Pred Percent Change: -0.024

Ticker: DKNG
Open: 42.66
Pred Close: 42.618
Pred Percent Change: -0.099

Ticker: CMCSA
Open: 57.52
Pred Close: 57.543
Pred Percent Change: 0.041

Ticker: PFE
Open: 39.68
Pred Close: 39.685
Pred Percent Change: 0.011


